In [1]:
from pymongo import MongoClient
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
%matplotlib inline

### get route info preped

In [9]:
client = MongoClient('mongodb://localhost:27017/')
db = client.routes
routes = db.routes
raw = routes.find()
df = pd.DataFrame(list(raw))
df_new = df.drop(['FA', '_id', 'average_rating', 'id', 'name', 'season', 'submitted_by', 'submitted_on', 'uiaa'], axis=1)
df_new.head(10)

,Aid,Alpine,Boulder,Ice,Mixed,Sport,TR,Trad,grade,height,original_grade,page_views,pitches
0,0,0,0,0,0,0,0,1,5.6,41.0,5.6,588,1.0
1,0,0,0,0,0,0,0,1,5.5,41.0,5.5,642,1.0
2,0,0,0,0,0,0,1,1,5.5,35.0,5.5,176,NaN
3,0,0,0,0,0,0,1,1,5.6,85.0,5.6,1263,1.0
4,0,0,0,0,0,0,1,1,5.6,30.0,5.6,678,1.0
5,0,0,0,0,0,1,0,0,5.7,40.0,5.7,326,NaN
6,0,0,0,0,0,0,1,1,5.7,95.0,5.7,454,1.0
7,0,0,0,0,0,1,0,0,5.5,95.0,5.5,18,NaN
8,0,0,0,0,0,0,0,1,5.7,NaN,5.7,51,NaN
9,0,0,0,0,0,1,0,0,5.3,75.0,5.3,39,NaN


In [10]:
df[df.duplicated(subset=['name', 'FA', 'grade', 'submitted_by', 'submitted_on'])].shape

(38, 22)

In [11]:
route_df = df.drop_duplicates(subset=['name', 'FA', 'grade', 'submitted_by', 'submitted_on'])

In [13]:
route_df_new = route_df.drop(['uiaa', 'submitted_on', 
                              'submitted_by', 'season', 
                              'name', 'id', 
                              'average_rating', '_id', 
                              'FA'], axis=1)

In [14]:
route_df_new.head()

,Aid,Alpine,Boulder,Ice,Mixed,Sport,TR,Trad,grade,height,original_grade,page_views,pitches
0,0,0,0,0,0,0,0,1,5.6,41.0,5.6,588,1.0
1,0,0,0,0,0,0,0,1,5.5,41.0,5.5,642,1.0
2,0,0,0,0,0,0,1,1,5.5,35.0,5.5,176,NaN
3,0,0,0,0,0,0,1,1,5.6,85.0,5.6,1263,1.0
4,0,0,0,0,0,0,1,1,5.6,30.0,5.6,678,1.0


In [15]:
route_df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3689 entries, 0 to 3726
Data columns (total 13 columns):
Aid               3689 non-null int64
Alpine            3689 non-null int64
Boulder           3689 non-null int64
Ice               3689 non-null int64
Mixed             3689 non-null int64
Sport             3689 non-null int64
TR                3689 non-null int64
Trad              3689 non-null int64
grade             3689 non-null object
height            2963 non-null float64
original_grade    3689 non-null object
page_views        3689 non-null int64
pitches           2106 non-null float64
dtypes: float64(2), int64(9), object(2)
memory usage: 403.5+ KB


In [16]:
df_grades = route_df_new[['grade','original_grade']]
df_dummies = pd.get_dummies(df_grades).drop('grade_3rd', axis=1)

In [17]:
concat_list = [route_df_new.drop(['grade','original_grade'], axis=1), df_dummies]
x_routes = pd.concat(concat_list, axis=1).fillna(route_df_new.mean())

In [88]:
x_routes.isnull().values.any()

False

### prep user info

In [22]:
db = client.users
users = db.users
raw = users.find()
users_df = pd.DataFrame(list(raw))
users_df.head().T

,0,1,2,3,4
_id,58d1912840b4417fa4b16db4,58d1912840b4417fa4b16db5,58d1912840b4417fa4b16db6,58d1912840b4417fa4b16db7,58d1912840b4417fa4b16db8
age,NaN,NaN,NaN,33,NaN
aid_follows,NaN,NaN,C5,NaN,NaN
aid_leads,NaN,NaN,C3,NaN,NaN
city,NaN,Seattle,Hawaii,Corvallis,NaN
compliments,0,1,10,0,0
favorite_climbs,leading trad is rad,"high mountain woody, fridge left, easy in an ...",none specified,outer space,none specified
female,NaN,0,0,0,NaN
ice_follows,NaN,NaN,NaN,NaN,NaN
ice_leads,NaN,NaN,NaN,NaN,NaN


In [64]:
users_df_new = users_df.drop(['name', '_id', 'other_interests', 'id', 'last_vist', 'favorite_climbs'], axis=1)
users_df_new.head(10).T

,0,1,2,3,4,5,6,7,8,9
age,NaN,NaN,NaN,33,NaN,44,NaN,25,26,NaN
aid_follows,NaN,NaN,C5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aid_leads,NaN,NaN,C3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city,NaN,Seattle,Hawaii,Corvallis,NaN,Fort Thomas,San Diego,albuquerque,Seattle,Redmond
compliments,0,1,10,0,0,0,2,0,0,0
female,NaN,0,0,0,NaN,0,1,1,0,0
ice_follows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WI4,NaN
ice_leads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WI3,NaN
likes_gym,0,1,1,1,0,1,0,1,0,1
likes_sport,0,1,1,1,0,1,0,1,0,1


In [65]:
users_df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2069 entries, 0 to 2068
Data columns (total 20 columns):
age              903 non-null float64
aid_follows      217 non-null object
aid_leads        217 non-null object
city             1155 non-null object
compliments      2069 non-null int64
female           1260 non-null float64
ice_follows      314 non-null object
ice_leads        314 non-null object
likes_gym        2069 non-null int64
likes_sport      2069 non-null int64
likes_tr         2069 non-null int64
likes_trad       2069 non-null int64
member_since     2069 non-null object
point_rank       1156 non-null float64
sport_follows    1036 non-null object
sport_lead       1036 non-null object
state            1133 non-null object
total_points     2069 non-null int64
trad_follows     998 non-null object
trad_lead        998 non-null object
dtypes: float64(3), int64(6), object(11)
memory usage: 323.4+ KB


In [66]:
# fill age with mean age
users_df_new['age'] = users_df_new['age'].fillna(users_df_new['age'].mean())

In [78]:
# fix point rank
users_df_new['point_rank'] = users_df_new['point_rank'].fillna(users_df_new['point_rank'].mean())

In [67]:
# fix datetime
users_df_new['member_since'] = pd.to_datetime(users_df_new['member_since'])
users_df_new.head().T

,0,1,2,3,4
age,33.2824,33.2824,33.2824,33,33.2824
aid_follows,NaN,NaN,C5,NaN,NaN
aid_leads,NaN,NaN,C3,NaN,NaN
city,NaN,Seattle,Hawaii,Corvallis,NaN
compliments,0,1,10,0,0
female,NaN,0,0,0,NaN
ice_follows,NaN,NaN,NaN,NaN,NaN
ice_leads,NaN,NaN,NaN,NaN,NaN
likes_gym,0,1,1,1,0
likes_sport,0,1,1,1,0


### time to make dummies

In [68]:
dummy_list1 = [u'aid_follows',
             u'aid_leads',
             u'ice_follows',
             u'ice_leads',
             u'sport_follows',
             u'sport_lead',
             u'trad_follows',
             u'trad_lead']
dummy_list2 = [u'city',u'state',]

In [69]:
df_subsets = users_df_new[dummy_list1]
dummy_df1 = pd.get_dummies(df_subsets)

In [70]:
df_subsets = users_df_new[dummy_list2]
dummy_df2 = pd.get_dummies(df_subsets)

In [75]:
dummy_df3 = pd.get_dummies(users_df_new['female']).rename( columns={0.0: 'male', 1.0: 'female'})

In [82]:
users_df_new.head().T
concat_list = [users_df_new.drop(dummy_list1+dummy_list2+['female'], axis=1), dummy_df1, dummy_df2, dummy_df3]
x_users = pd.concat(concat_list, axis=1)

In [83]:
x_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2069 entries, 0 to 2068
Columns: 599 entries, age to female
dtypes: datetime64[ns](1), float64(2), int64(6), uint8(590)
memory usage: 1.3 MB


In [87]:
x_users.isnull().values.any()

False

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from urllib import urlencode
import pandas as pd
from pymongo import MongoClient


def scrape_route_page(html):
    '''
    Scrapes route pages for html 
    returns soup, url for the rating page for the route, and route name
    '''
    soup = BeautifulSoup(html, 'html.parser')
    # convert to string from unicode
    star_url = str(soup.find('span', {'id': 'starSummaryText'}).\
                find('a', href=True).get('href'))
    page_tag = soup.find('div', {'id': 'rspCol800'})
    route_name = page_tag.find('span', {'itemprop': 'itemreviewed'}).text
    return star_url, html, route_name


def scrape_ratings_by_user(html):
    '''
    Input: query to ratings page and route_name
    Output: user_url, and rating_info
    '''
    rating_dict = {'username': [], 'rating': [], 'user_url': []}
    rating_dict['html'] = html
    soup = BeautifulSoup(html, 'html.parser')
    soup.find('div', {'id': 'navBox'})
    a = soup.find('div', {'id': 'navBox'})
    url = a.findAll('a', href=True)[-1].get('href')
    rating_dict['route_url'] = url
    table_tag = soup.findAll('table')
    # the 4th table is the one with star votes
    for row in table_tag[3].findAll('tr'):
        for i, column in enumerate(row.findAll('td')):
            if i % 2 == 0:
                rating_dict['username'].append(column.text)  # username
                rating_dict['user_url'].append(str(column.find('a', href=True).\
                            get('href')))  # query for user url
            if i % 2 == 1:
                rating_dict['rating'].append(\
                    int(column.text.split('Html(')[1][0]) - 1) # number of stars
    return rating_dict


client = MongoClient('mongodb://localhost:27017/')
db = client.route_url
route_url = db.route_url
route_url_dict = route_url.find()
route_href_list = list(route_url_dict)[0]['route_urls']

db = client.route_html_collection
route_html_collection = db.route_html_collection
raw_data = route_html_collection.find()
df_route_html = pd.DataFrame(list(raw_data))

db = client.ratings_info
ratings_info = db.ratings_info
raw_data = ratings_info.find()
ratings_df = pd.DataFrame(list(raw_data))
'''
route_list = []
for route_url,route_html in zip(route_href_list, df_route_html['html']):
    star_url, route_html, route_name = scrape_route_page(route_html)
    route_html_dict = {'route': route_name, 
                        'html': route_html,
                        'url': route_url}
    route_list.append(route_html_dict)
'''
ratings_list = [] 
for ratings_html, route_name in zip(ratings_df['html'], ratings_df['route']):
    rating_dict = scrape_ratings_by_user(ratings_html)
    rating_dict['route'] = route_name
    ratings_list.append(rating_dict)


In [3]:
def add_to_rating_database(rating_dict):
    client = MongoClient('mongodb://localhost:27017/')
    db = client.ratings_collection_updated
    ratings_collection = db.ratings_collection
    try:
        ratings_collection.insert_one(rating_dict)
    except:
        print("add_to_rating_database error: ", rating_dict.keys())

for ratings in ratings_list:
    add_to_rating_database(ratings)

In [ ]:
client = MongoClient('mongodb://localhost:27017/')
db = cclient.ratings_collection_updated
ratings_collection = db.ratings_collection
raw_data = ratings_collection.find()
route_href_list = list(route_url_dict)[0]['route_urls']